<a href="https://colab.research.google.com/github/keisyashakila/MSCI-151/blob/main/MILP_CW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from collections import defaultdict
import os
import csv

Correcting import statements:
- `from collection import dafaultdict` to `from collections import defaultdict`
- Assuming `import css` was intended as `import csv` as `css` is not a standard Python module.

In [32]:
from collections import defaultdict
import os
import csv

In [33]:
!pip install pulp

In [34]:
import pulp as pl

In [35]:
baristas = ["Max", "Jiwa", "Fore", "Donna", "Paul"]
days = list(range(1,7))
blocks = list(range(1,5))
H = 4

days 1-7, due to 7 days a week

In [36]:
cost = {"Max": 50000, "Jiwa": 50000, "Fore": 50000, "Donna": 150000, "Paul": 150000}
etype = {"Max": "P", "Jiwa": "P", "Fore": "P", "Donna": "F", "Paul": "F"}
minWeekly_default = {"Max": 12, "Jiwa": 12, "Fore": 12, "Donna": 36, "Paul": 36}

In [37]:
avail = {("Max", 1): 8, ("Max", 2): 8, ("Max", 3): 8, ("Max", 4): 0, ("Max", 5): 8, ("Max", 6): 4, ("Max", 7): 4, ("Jiwa", 1): 4, ("Jiwa", 2): 4, ("Jiwa", 3): 8, ("Jiwa", 4): 8, ("Jiwa", 5): 0, ("Jiwa", 6): 4, ("Jiwa", 7): 0, ("Fore", 1): 8, ("Fore", 2): 0, ("Fore", 3): 8, ("Fore", 4): 8, ("Fore", 5): 8, ("Fore", 6): 8, ("Fore", 7): 0, ("Donna", 1): 12, ("Donna", 2): 12, ("Donna", 3): 12, ("Donna", 4): 12, ("Donna", 5): 12, ("Donna", 6): 12, ("Donna", 7): 8, ("Paul", 1): 12, ("Paul", 2): 8, ("Paul", 3): 12, ("Paul", 4): 12, ("Paul", 5): 8, ("Paul", 6): 12, ("Paul", 7): 12}

In [38]:
days = {1: "Monday", 2: "Tuesday", 3: "Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}

In [39]:
blocks = {1: "07:00-11:00", 2: "11:00-15:00", 3: "15:00-19:00", 4: "19:00-23:00"}
block_ids = list(blocks.keys())

In [40]:
def print_schedule(y_vars):
    for d in days:
        print(f"\n{day_names[d]}")
        for b in block_ids:
            assigned = [o for o in baristas if y_vars[o, d, b].varValue == 1]
            print(f"  Block {b} ({blocks[b]}): {', '.join(assigned) if assigned else 'None'}")

mathematical model

In [41]:
model = pl.LpProblem("Barista_Scheduling", pl.LpMinimize)

decision variable ​￼

In [42]:
y = pl.LpVariable.dicts(
    "y",
    ((o, d, b) for o in baristas for d in days for b in blocks),
    cat="Binary"
)

Minimise Cost

In [43]:
model += pl.lpSum(H * cost[o] * y[o, d, b] for o in baristas for d in days for b in blocks)

# **Constraints**

Coverage (one barista per block)

In [44]:
for d in days:
    for b in blocks:
        model += pl.lpSum(y[o, d, b] for o in baristas) >= 1

Per‑day availability (hours)

In [45]:
for o in baristas:
    for d in days:
        model += H * pl.lpSum(y[o, d, b] for b in blocks) <= avail[(o, d)]

Daily block limit

In [46]:
for o in baristas:
    for d in days:
        if etype[o] == "P":  # Part-time
            model += pl.lpSum(y[o, d, b] for b in blocks) <= 2
        if etype[o] == "F":  # Full-time
            model += pl.lpSum(y[o, d, b] for b in blocks) <= 3

Weekly minimum hours

In [47]:
for o in baristas:
    model += H * pl.lpSum(y[o, d, b] for d in days for b in blocks) >= minWeekly_default[o]

Binary variables (the problem)

*   **cat="Binary"** part automatically enforces that each variable is either 0 or 1.

## **Big Picture**

•  Coverage → shop never empty.

•  Per-day Availability → respect daily limits.

•  Daily limits → enforce part/full-time rules.

•  Weekly minimums → guarantee fair hours.

•  Binary variables → clean yes/no assignments.

---
This show each assignment of a barista to a block on a day as yes/no. Then, it will choose which are yes (1) and which are no (0)


# **Solving part**

Solve the Model

In [49]:
model.solve()

total_calculated_cost = 0
print("Individual Barista Costs:")
for o in baristas:
    # Get total hours for the barista
    hours_worked = sum(H * y[o, d, b].varValue for d in days for b in block_ids)

    # Get hourly wage for the barista
    hourly_wage = cost[o]

    # Calculate individual cost
    barista_cost = hours_worked * hourly_wage
    total_calculated_cost += barista_cost
    print(f"  {o}: {hours_worked} hours * {hourly_wage} (hourly wage) = {barista_cost}")

print(f"\nTotal Calculated Cost: {total_calculated_cost}")
print(f"Total Cost from Model Objective: {pl.value(model.objective)}")

if total_calculated_cost == pl.value(model.objective):
    print("The calculated total cost matches the model's objective value.")
else:
    print("There is a discrepancy between the calculated total cost and the model's objective value.")

Individual Barista Costs:
  Max: 12.0 hours * 50000 (hourly wage) = 600000.0
  Jiwa: 12.0 hours * 50000 (hourly wage) = 600000.0
  Fore: 16.0 hours * 50000 (hourly wage) = 800000.0
  Donna: 36.0 hours * 150000 (hourly wage) = 5400000.0
  Paul: 36.0 hours * 150000 (hourly wage) = 5400000.0

Total Calculated Cost: 12800000.0
Total Cost from Model Objective: 12800000.0
The calculated total cost matches the model's objective value.


Fore works for 16 hours due to:
*   cheaper than full-timers
*   all rules are still respecting all contraints.
*   Fore has 8 hours available on most weekdays



Weekly Hours per Barista

In [52]:
print("Weekly Hours per Barista:")
for o in baristas:
    total_weekly_hours = sum(H * y[o, d, b].varValue for d in days for b in block_ids)
    print(f"  {o}: {total_weekly_hours} hours")

Weekly Hours per Barista:
  Max: 12.0 hours
  Jiwa: 12.0 hours
  Fore: 16.0 hours
  Donna: 36.0 hours
  Paul: 36.0 hours


Daily Schedule (Readable)

In [56]:
def print_schedule(y_vars):
    for d_id, d_name in days.items():
        print(f"\n{d_name}")
        for b_id in block_ids:
            assigned = [o for o in baristas if y_vars[o, d_id, b_id].varValue == 1]
            print(f"  Block {b_id} ({blocks[b_id]}): {', '.join(assigned) if assigned else 'None'}")

print_schedule(y)


Monday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Paul
  Block 3 (15:00-19:00): Max
  Block 4 (19:00-23:00): Jiwa

Tuesday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Donna
  Block 3 (15:00-19:00): Jiwa
  Block 4 (19:00-23:00): Max

Wednesday
  Block 1 (07:00-11:00): Paul
  Block 2 (11:00-15:00): Fore
  Block 3 (15:00-19:00): Fore
  Block 4 (19:00-23:00): Paul

Thursday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Paul
  Block 3 (15:00-19:00): Donna
  Block 4 (19:00-23:00): Paul

Friday
  Block 1 (07:00-11:00): Paul
  Block 2 (11:00-15:00): Fore
  Block 3 (15:00-19:00): Fore
  Block 4 (19:00-23:00): Paul

Saturday
  Block 1 (07:00-11:00): Donna
  Block 2 (11:00-15:00): Jiwa
  Block 3 (15:00-19:00): Donna
  Block 4 (19:00-23:00): Max

Sunday
  Block 1 (07:00-11:00): Paul
  Block 2 (11:00-15:00): Donna
  Block 3 (15:00-19:00): Paul
  Block 4 (19:00-23:00): Donna
